In [1]:
import pandas as pd
import datapreprocessing

In [2]:
filepath ='../src/dataset/DataSet.csv'
dataset = pd.read_csv(filepath)

In [3]:
dataset.head()

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,in_balanced_dataset
0,Marketing Intern,"US, NY, New York",Marketing,NaN,"<h3>We're Food52, and we've created a groundbr...","<p>Food52, a fast-growing, James Beard Award-w...",<ul>\r\n<li>Experience with content management...,NaN,f,t,f,Other,Internship,NaN,NaN,Marketing,f,f
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"<h3>90 Seconds, the worlds Cloud Video Product...",<p>Organised - Focused - Vibrant - Awesome!<br...,<p><b>What we expect from you:</b></p>\r\n<p>Y...,<h3><b>What you will get from us</b></h3>\r\n<...,f,t,f,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,f,f
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,<h3></h3>\r\n<p>Valor Services provides Workfo...,"<p>Our client, located in Houston, is actively...",<ul>\r\n<li>Implement pre-commissioning and co...,NaN,f,t,f,NaN,NaN,NaN,NaN,NaN,f,f
3,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,<p>Our passion for improving quality of life t...,<p><b>THE COMPANY: ESRI – Environmental System...,<ul>\r\n<li>\r\n<b>EDUCATION: </b>Bachelor’s o...,<p>Our culture is anything but corporate—we ha...,f,t,f,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,f,f
4,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,<p>SpotSource Solutions LLC is a Global Human ...,<p><b>JOB TITLE:</b> Itemization Review Manage...,<p><b>QUALIFICATIONS:</b></p>\r\n<ul>\r\n<li>R...,<p>Full Benefits Offered</p>,f,t,t,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,f,f


In [4]:
dataset = datapreprocessing.datasetprocess(dataset)

In [5]:
dataset.head()

,fraudulent,text
0,0,market intern marketingwer food weve creat gro...
1,0,custom servic cloud video product success seco...
2,0,commiss machineri assist cmavalor servic provi...
3,0,account execut washington dc salesour passion ...
4,0,bill review managerspotsourc solut llc global ...


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split

# 使用TF-IDF表示文本特征
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(dataset['text'])

# 使用SMOTETomek技术进行过采样和欠采样
smt = SMOTETomek(random_state=42)
X_resampled, y_resampled = smt.fit_resample(X_tfidf, dataset['fraudulent'])

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from mlxtend.classifier import StackingCVClassifier

# 定义基础模型
base_models = [
    ('nb', MultinomialNB()),
    ('rf', RandomForestClassifier(n_estimators=100, class_weight='balanced')),
    ('svm', SVC(kernel='linear', class_weight='balanced', probability=True))
]

# 定义元模型
meta_model = LogisticRegression()

# 定义Stacking模型
stacking_model = StackingCVClassifier(classifiers=[model for _, model in base_models],
                                      meta_classifier=meta_model,
                                      cv=StratifiedKFold(n_splits=4),
                                      use_probas=True,
                                      verbose=2)

stacking_model.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = stacking_model.predict(X_test)

# 打印分类报告
print(classification_report(y_test, y_pred))

Fitting 3 classifiers...
Fitting classifier1: multinomialnb (1/3)
MultinomialNB()


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting classifier2: randomforestclassifier (2/3)
RandomForestClassifier(class_weight='balanced')


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  7.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting classifier3: svc (3/3)
SVC(class_weight='balanced', kernel='linear', probability=True)


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.2min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 33.1min finished


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3157
           1       1.00      1.00      1.00      3087

    accuracy                           1.00      6244
   macro avg       1.00      1.00      1.00      6244
weighted avg       1.00      1.00      1.00      6244



In [21]:
import joblib

# 导出Stacking模型
joblib.dump(stacking_model, '../static/model/stacking_model.pkl')

# 导出基础模型
for name, model in base_models:
    joblib.dump(model, f'../static/model/{name}_model.pkl')

# 导出向量化器
joblib.dump(tfidf_vectorizer, '../static/vectorizer/tfidfvectorizer.pkl')

['../static/vectorizer/tfidfvectorizer.pkl']